In [ ]:
from maths import abs

def find_seed(g, c=0, eps=2**(-26)):
    
    """On réalise une dichotomie pour trouver une racine de g(t)-c"""
    
    a = 0
    b = 1
    t = (a+b)/2
    
    if g(0) > g(1):
        if c > g(0) or c < g(1):
            return(None)
        
        while abs(g(t)-c) > eps:
            if g(t) > c:
                a = t
            else:
                b = t
            t = (a+b)/2

    else :
        if c < g(0) or c > g(1):
            return(None)
        
        while abs(g(t)-c) > eps:
            if g(t) > c:
                b = t
            else:
                a = t
            t = (a+b)/2
            
    return(t)

In [ ]:
def simple_contour(f, c=0.0, delta=0.01):
    X = []
    Y = []
    
    for x in range(0,1,delta):
        g = lambda t: f(x,t)
        t = find_seed(g,c,2**(-26))
        
        if t == None:
            return([],[])
        else:
            X.append(x)
            Y.append(y)
    return(X,Y)